In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import sklearn.metrics as sm
from sklearn.model_selection import cross_validate
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import GradientBoostingClassifier as grad
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
import scipy.stats as stats
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, "../util/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Adding dummies and train-test-split

In [2]:
datingTrainExists = exists('../data/processedData/datingTrain.csv')
datingTestExists = exists('../data/processedData/datingTest.csv')
datingFullExists = exists('../data/processedData/datingFull.csv')

In [3]:
if exists('../data/processedData/columnDataDictionary.json'):
    with open('../data/processedData/columnDataDictionary.json') as d:
        columnDataDictionary = json.load(d)
else:
    with open('../data/columnDataDictionary.json') as d:
        columnDataDictionary = json.load(d)
columnList = columnDataDictionary['columnList']
nonBinaryCategoricalList = columnDataDictionary['nonBinaryCategoricalList']
stringToFloatList = columnDataDictionary['stringToFloatList']
pointDistributionList = columnDataDictionary['pointDistributionList']
partnerList = columnDataDictionary['partnerList']

In [4]:
if (datingTrainExists and datingTestExists and datingFullExists):
    datingTrain = pd.read_csv('../data/processedData/datingTrain.csv')
    datingTest = pd.read_csv('../data/processedData/datingTest.csv')
    datingFull = pd.read_csv('../data/processedData/datingFull.csv')
    
    for df in [datingTrain,datingTest,datingFull]:
        df['zipcode'] = df['zipcode'].apply(str)
        if 'zipcode_o' in list(df.columns):
            df['zipcode_o'] = df['zipcode_o'].apply(str)
        for col in nonBinaryCategoricalList:
            if col in list(df.columns):
                df[col] = df[col].apply(str)
        
else:
    !rm -r ../data/processedData
    !mkdir ../data/processedData
    
    datingData = pd.read_csv('../data/encoded-SpeedDatingData-WithLocations.csv')   
    
    blindDateData = datingData[columnList]
    
    for col in stringToFloatList:
        blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
    
    blindDateData['zipcode'] = blindDateData['zipcode'].apply(str)
    blindDateData['zipcode'] = blindDateData['zipcode'].str.replace(',', '')
    
    for col in nonBinaryCategoricalList:
        blindDateData[col] = blindDateData[col].apply(str)
    
    blindDateCategoricalData = blindDateData.select_dtypes(include=['O'])
    for col in blindDateCategoricalData.columns:
        blindDateData[col]=blindDateData[col].fillna('nan')
    
    for col in blindDateCategoricalData.columns:
        dummyData = pd.get_dummies(blindDateData[col],prefix=col,drop_first=True)
        if len(dummyData.columns) <= 25:
            for dummyCol in dummyData.columns:
                if col in partnerList:
                    partnerList.append(str(dummyCol))
                    partnerDummies = [partnerdummy+"_o" for partnerdummy in list(dummyData.columns)]
            blindDateData = pd.concat([blindDateData,dummyData],axis=1)
    
    blindDateCols = blindDateData.columns
    identityList = ["iid","pid"]
    partnerList = [col for col in columnList if ((col not in identityList) and (col != "match") and ("_o" not in col))]
    partnerList += [col for col in blindDateData.columns if "go_out" in col]
    
    partnerList = list(set(partnerList))
    columnDataDictionary = {"columnList": columnList,
                        "nonBinaryCategoricalList": nonBinaryCategoricalList,
                        "stringToFloatList": stringToFloatList,
                        "pointDistributionList": pointDistributionList,
                        "partnerList": partnerList}

    with open('../data/processedData/columnDataDictionary.json', 'w') as fp:
            json.dump(columnDataDictionary, fp)
    
    datingFull = blindDateData.copy()
    match = datingFull['match']
    X = datingFull.drop(['match'], axis=1)
    
    datingTrain, datingTest, matchTrain, matchTest = train_test_split(X, match, test_size=0.2)
    
    datingTrain['match'] = matchTrain
    datingTest['match'] = matchTest
    
    datingTrain.to_csv('../data/processedData/datingTrain.csv',index=False)
    datingTest.to_csv('../data/processedData/datingTest.csv',index=False)
    datingFull.to_csv('../data/processedData/datingFull.csv',index=False)

<ipython-input-4-fec425ccdf82>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
<ipython-input-4-fec425ccdf82>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blindDateData['zipcode'] = blindDateData['zipcode'].apply(str)
<ipython-input-4-fec425ccdf82>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

## Data Cleaning

### Join partner

In [5]:
if ("iid" in datingTrain.columns) or ("pid" in datingTrain.columns):
    partner = datingFull.copy()
    datingTrain = util.joinToPartner(datingTrain,partner).drop(["iid","pid","iid_o","pid_o"],axis=1)
    datingTrain.to_csv('../data/processedData/datingTrain.csv',index=False)

../util/util.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['iid_o'] = partner_o['iid']
../util/util.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['pid_o'] = partner_o['pid']


### Get distance

In [6]:
if "partnerDistance" not in datingTrain.columns:
    datingTrain = util.returnDFWithpartnerDistance(datingTrain,"train",True)
    datingTrain.to_csv('../data/processedData/datingTrain.csv',index=False)

21:36:27
6701 of 6702: 99.98507908087139% complete
2.58131900925097% of data is None


### Fix ambiguous scores

In [7]:
halfwayChangeColumns = [str(col) for col in datingTrain.columns if (("1_s" in str(col)) | ("3_s" in str(col)))]

if(len(halfwayChangeColumns) > 0):
    datingTrain = util.fixAmbiguousScores(datingTrain)
    util.halfwayQuestionSanityTest(datingTrain," post-fixAmbiguousScores and pre-saving")
    datingTrain.to_csv('../data/processedData/datingTrain.csv',index=False)

### Replace Nans

In [8]:
datingTrainNumerical = datingTrain.select_dtypes(include=['uint8','int64','float64']).drop("match",axis=1)

if (exists('../data/processedData/trainNanReplacementValuesDictionary.json')):
    with open('../data/processedData/trainNanReplacementValuesDictionary.json') as d:
        trainNanReplacementValuesDictionary = json.load(d)
else:
    trainNanReplacementValuesDictionary = {}
    for col in datingTrainNumerical:   
        if len(list(set(datingTrainNumerical[col]))) <=30:
            setValues = [val for val in list(set(datingTrainNumerical[col])) if ((np.isnan(val) == False) and np.isfinite(val))]
            setValues = pd.Series(setValues)
            trainNanReplacementValuesDictionary[str(col)] = round(np.mean(setValues.mode().values))
        else:
            trainNanReplacementValuesDictionary[str(col)] = datingTrainNumerical[col].mean()
    with open('../data/processedData/trainNanReplacementValuesDictionary.json', 'w') as fp:
        json.dump(trainNanReplacementValuesDictionary, fp)

In [9]:
match = datingTrain["match"]
X = util.replaceNansWithTrainingDataValues(datingTrainNumerical)

# Training

In [10]:
sqrtn = int(np.sqrt(X.shape[0]))
sqrtfeatures = int(np.sqrt(X.shape[1]))
log2features = int(np.log2(X.shape[1]))
geometricTerms = sorted([np.sqrt(X.shape[1]),np.log2(X.shape[1])])
thirdGeometricTerm = int(geometricTerms[1] * (geometricTerms[1]/geometricTerms[0]))

## Logistic Regression

In [11]:
logModel = lm.LogisticRegression(max_iter=1e9)
logPipe = make_pipeline(StandardScaler(), logModel)
logPipe.fit(X,match)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000000000.0))])

## KNN

In [12]:
knn5 = knn(n_neighbors=5)
knnsqrtn = knn(n_neighbors=sqrtn)
knn5.fit(X,match)
knnsqrtn.fit(X,match)

KNeighborsClassifier(n_neighbors=81)

## Boosting

In [13]:
gradientdeci = grad(learning_rate=0.1)
gradientdeka = grad(learning_rate=10)

gradientdeci.fit(X,match)
gradientdeka.fit(X,match)

GradientBoostingClassifier(learning_rate=10)

## Random Forest Classifier

In [14]:
if exists("../data/processedData/forestParams.json"):
    with open('../data/processedData/forestParams.json') as d:
        forestParams = json.load(d)
        preciseForestParams = forestParams["preciseForestParams"]
        recallForestParams = forestParams["recallForestParams"]
else:
    print(f"sqrt(nFeatures) ~ {sqrtfeatures}")
    print(f"log2(nFeatures) ~ {log2features}")
    print(f"thirdGeometricTerm ~ {thirdGeometricTerm}")
    searchParams = {
        "criterion": ["gini","entropy","log_loss"],
        "n_estimators": [100,200,300],
        "max_depth":[sqrtfeatures,log2features,thirdGeometricTerm],
        "max_features":[sqrtfeatures,log2features,thirdGeometricTerm]
    }

    preciseForest0 = rf()
    recallForest0 = rf()
    
    recallForestGrid = ms.GridSearchCV(recallForest0, param_grid=searchParams, scoring='recall',n_jobs=9)
    recallForestGrid.fit(X,match)
    recallForestParams = recallForestGrid.best_params_
    print("recall params:")
    print(recallForestParams)

    for key in searchParams.keys():
        searchParams[key] = [val for val in searchParams[key] if val != recallForestParams[key]]

    preciseForestGrid = ms.GridSearchCV(preciseForest0, param_grid=searchParams, scoring='precision',n_jobs=9)
    preciseForestGrid.fit(X,match)
    preciseForestParams = preciseForestGrid.best_params_
    print("precision params:")
    print(preciseForestParams)


    forestParams = {
        "preciseForestParams": preciseForestParams,
        "recallForestParams": recallForestParams
    }
    with open("../data/processedData/forestParams.json", 'w') as fp:
        json.dump(forestParams, fp)
    
if exists("../data/processedData/forestParams.json"):
    preciseForest = rf(n_estimators = preciseForestParams["n_estimators"],
                    criterion = preciseForestParams["criterion"],
                    max_depth = preciseForestParams["max_depth"],
                    max_features = preciseForestParams["max_features"])
    recallForest = rf(n_estimators = recallForestParams["n_estimators"],
                    criterion = recallForestParams["criterion"],
                    max_depth = recallForestParams["max_depth"],
                    max_features = recallForestParams["max_features"])

    preciseForest.fit(X,match)
    recallForest.fit(X,match)

sqrt(nFeatures) ~ 13
log2(nFeatures) ~ 7
thirdGeometricTerm ~ 24
recall params:
{'criterion': 'gini', 'max_depth': 24, 'max_features': 24, 'n_estimators': 100}
precision params:
{'criterion': 'log_loss', 'max_depth': 13, 'max_features': 7, 'n_estimators': 300}


# Top 10 Feature Importances

In [15]:
XColumns = list(X.columns)

## Logistic Regression

In [16]:
coefficients = [coef for coef in logPipe.named_steps['logisticregression'].coef_.reshape(-1,)]
absCoefficients = [absCoef for absCoef in np.abs(np.array(coefficients))]

logImportances = pd.DataFrame({
    "feature": XColumns,
    "coefficients": coefficients,
    "absCoefficients": absCoefficients},columns = ["feature","coefficients","absCoefficients"])
logImportancesSorted = logImportances.sort_values(by="absCoefficients", ascending=False)
print(f'logistic regression top 10 feature importances')
for i in range(10):
    featureRow = logImportancesSorted.iloc[i]
    feature = featureRow['feature']
    featureValue = featureRow['coefficients']
    print(f'Rank {i}: {feature}: score: {featureValue}')

logistic regression top 10 feature importances
Rank 0: pf_o_att: score: -0.34382742479413164
Rank 1: match_es_o: score: 0.3050481776209463
Rank 2: field_cd_nan: score: -0.3005992400335028
Rank 3: race_o_2.0: score: -0.29062428110539157
Rank 4: match_es: score: 0.2831234499996134
Rank 5: race_2.0: score: -0.2643145209381777
Rank 6: race_nan: score: 0.25219727286491816
Rank 7: attr1_1: score: -0.2512990385616912
Rank 8: pf_o_fun: score: 0.24522311139339745
Rank 9: race_o_4.0: score: -0.23179805769126866


## KNN focuses on nearest neighbors, not on specific features

## Gradient Boosting & Random Forest Classifier

In [17]:
models = [gradientdeci,gradientdeka,preciseForest,recallForest]
modelNames = ["gradientdeci","gradientdeka","preciseForest","recallForest"]

for i in range(4):
    util.displayFeatureImportances(XColumns,models[i],modelNames[i])

gradientdeci top 10 feature importances
Rank 0: match_es: score: 0.09584576370771493
Rank 1: match_es_o: score: 0.0929749962214017
Rank 2: shar1_1: score: 0.03700506815830845
Rank 3: shar1_1_o: score: 0.03671792478533394
Rank 4: lats_o: score: 0.026205447623322855
Rank 5: order_o: score: 0.02535045482310188
Rank 6: fun5_1_o: score: 0.02117978168104133
Rank 7: attr1_1: score: 0.01935798140590371
Rank 8: fun3_1_o: score: 0.01930718057360298
Rank 9: partnerDistance: score: 0.019252067999163097


gradientdeka top 10 feature importances
Rank 0: match_es: score: 0.38002011698990834
Rank 1: shar1_1_o: score: 0.33251084324151564
Rank 2: match_es_o: score: 0.2824233925012947
Rank 3: sinc4_1: score: 0.0026370402351203843
Rank 4: partnerDistance: score: 0.0017485013455022266
Rank 5: expnum_o: score: 0.0002775830625245308
Rank 6: intel5_1: score: 0.00013590154729871222
Rank 7: fun3_1_o: score: 0.00013147761717434092
Rank 8: pf_o_att: score: 8.227757012498585e-05
Rank 9: order_o: score: 2.328497548

# Individual Testing

In [18]:
if ("iid" in datingTest.columns) or ("pid" in datingTest.columns):
    partner = datingFull.copy()
    datingTest = util.joinToPartner(datingTest,partner).drop(["iid","pid","iid_o","pid_o"],axis=1)

if "partnerDistance" not in datingTest.columns:
    datingTest = util.returnDFWithpartnerDistance(datingTest,"test",True)

halfwayChangeColumns = [str(col) for col in datingTest.columns if (("1_s" in str(col)) | ("3_s" in str(col)))]

if(len(halfwayChangeColumns) > 0):
    datingTest = util.fixAmbiguousScores(datingTest)
    util.halfwayQuestionSanityTest(datingTest," post-fixAmbiguousScores and pre-saving")

datingTestNumerical = datingTest.select_dtypes(include=['uint8','int64','float64']).drop("match",axis=1)
matchTest = datingTest["match"]
XTest = util.replaceNansWithTrainingDataValues(datingTestNumerical)

../util/util.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['iid_o'] = partner_o['iid']
../util/util.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['pid_o'] = partner_o['pid']


21:46:24
1675 of 1676: 99.94033412887828% complete
1.9689737470167064% of data is None


## Logistic Regression

In [19]:
yPredict = logPipe.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"logPipe")

logPipe accuracy score: 0.837708830548926
logPipe recall score: 0.07720588235294118
logPipe precision score: 0.5


## KNN

In [20]:
yPredict = knn5.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"knn5")
yPredict = knnsqrtn.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"knnsqrtn")

knn5 accuracy score: 0.8198090692124105
knn5 recall score: 0.125
knn5 precision score: 0.3469387755102041
knnsqrtn accuracy score: 0.837708830548926
knnsqrtn recall score: 0.0
knnsqrtn precision score: 0.0


/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Boosting

In [21]:
yPredict = gradientdeci.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"gradientdeci")
yPredict = gradientdeka.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"gradientdeka")

gradientdeci accuracy score: 0.844272076372315
gradientdeci recall score: 0.07352941176470588
gradientdeci precision score: 0.6896551724137931
gradientdeka accuracy score: 0.752983293556086
gradientdeka recall score: 0.23161764705882354
gradientdeka precision score: 0.23507462686567165


## Random Forest Classifier

In [22]:
yPredict = preciseForest.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"preciseForest")
yPredict = recallForest.predict(XTest)
util.displayMetricScores(matchTest,yPredict,"recallForest")

preciseForest accuracy score: 0.8400954653937948
preciseForest recall score: 0.014705882352941176
preciseForest precision score: 1.0
recallForest accuracy score: 0.837708830548926
recallForest recall score: 0.051470588235294115
recallForest precision score: 0.5


# Ensemble Testing

In [23]:
logModel = lm.LogisticRegression(max_iter=1e9)
logPipe = make_pipeline(StandardScaler(), logModel)
knn5 = knn(n_neighbors=5)
knnsqrtn = knn(n_neighbors=sqrtn)
gradientdeci = grad(learning_rate=0.1)
gradientdeka = grad(learning_rate=10)
preciseForest = rf(n_estimators = preciseForestParams["n_estimators"],
                    criterion = preciseForestParams["criterion"],
                    max_depth = preciseForestParams["max_depth"],
                    max_features = preciseForestParams["max_features"])
recallForest = rf(n_estimators = recallForestParams["n_estimators"],
                  criterion = recallForestParams["criterion"],
                  max_depth = recallForestParams["max_depth"],
                  max_features = recallForestParams["max_features"])

ensembleVote = VotingClassifier(
    estimators = [
        ("logModel",logPipe),
        ("knn5",knn5),
        ("knnsqrtn",knnsqrtn),
        ("gradientdeci",gradientdeci),
        ("gradientdeka",gradientdeka),
        ("preciseForest",preciseForest),
        ("recallForest",recallForest)
    ]
)

ensembleVote.fit(X,match)
ensembleDecision = ensembleVote.predict(XTest)
util.displayMetricScores(matchTest,ensembleDecision,"Ensemble")

Ensemble accuracy score: 0.8394988066825776
Ensemble recall score: 0.014705882352941176
Ensemble precision score: 0.8
